## Setting the Database

### Loading DataFrame & Change to desired input format for DB

In [ ]:
## loading the data 
import pandas as pd
## defining the path 
PATH = "./data/clean_data.csv"
## loading 
df = pd.read_csv(PATH)
## reseting indeces
dfa = df.reset_index(drop=True)

## number of materials 
mats = [x for x in df.columns if "M-" in x]
n_mats = len(mats)
print(n_mats)

In [ ]:
## current dataframe: Processes | Material 1| Material 2| Material 3| ... 
## convert to Processes | Materials | Values
long_df = pd.wide_to_long(dfa, ["M-"], i="Processes", j="Material").reset_index().rename(columns={"M-": "Consumption"})
## add Input or Output column
## x > 0 -> Input, x < 0 -> Output, x = 0 -> None
long_df['IO'] = long_df['Consumption'].apply(lambda x: 1 if x > 0 else -1 if x < 0 else 0)
## adding the text to it in case
long_df['IO_txt'] = long_df['IO'].apply(lambda x: "Input" if x == 1 else "Output" if x == -1 else "None")
long_df

In [ ]:
## the dataframe is per process, 
long_df = long_df.sort_values(by=['Processes','Material'])
long_df_nz = long_df.query("Consumption != 0")

In [ ]:
long_df

In [ ]:
## Used Materials
input_materials = long_df_nz.query("IO == 1")
used_materials_per_process = input_materials.groupby("Processes")['Material'].apply(list).reset_index()
used_materials_per_process.rename(columns={"Material": "Inputs"}, inplace=True)
## Energy to Input 
used_consumption_materials_per_process = input_materials.groupby("Processes")['Consumption'].apply(list).reset_index()
used_consumption_materials_per_process.rename(columns={"Consumption": "InputCost"}, inplace=True)
## merge 
used_materials_df = input_materials.merge(used_materials_per_process, on="Processes", how="left")
used_materials_df = used_materials_df.merge(used_consumption_materials_per_process, on="Processes", how="left")


## Produced materials
output_materials = long_df_nz.query("IO == -1")
produced_materials_per_process = output_materials.groupby("Processes")['Material'].apply(list).reset_index()
produced_materials_per_process.rename(columns={"Material": "Outputs"}, inplace=True)
## Consumption Needed
produced_consumption_materials_per_process = output_materials.groupby("Processes")['Consumption'].apply(list).reset_index()
produced_consumption_materials_per_process.rename(columns={"Consumption": "OutputCost"}, inplace=True)
## merge 
produced_materials_df = output_materials.merge(produced_materials_per_process, on="Processes", how="left")
produced_materials_df = produced_materials_df.merge(produced_consumption_materials_per_process, on="Processes", how="left")
produced_materials_df


## tmp1 : used_materials drop Material	Consumption	IO	IO_txt
tmp1 = used_materials_df.drop(columns=["Material", "Consumption", "IO", "IO_txt"])
## tmp2: produced_materials_df
tmp2 = produced_materials_df.drop(columns=["Material", "Consumption", "IO", "IO_txt"])
tmp3 = tmp1.merge(tmp2, on="Processes", how="left")

# output_cost = lambda x: x if x>0 else 0
# input_cost = lambda x: x if x<0 else 0
# ## map the function to the list
tmp3['TotalOutputCost'] = tmp3['OutputCost'].apply(lambda x: sum(x))
tmp3['TotalInputCost'] = tmp3['InputCost'].apply(lambda x: sum(x))
## drop the duplicates 
tmp3 = tmp3.drop_duplicates('Processes')
tmp3.head()


In [ ]:
long_df_nz.drop(columns=['IO','IO_txt'])

### Start the Database

Essentially doing this 

1) Database 
2) Connection to DB with action 
3) Ask queries to the system

In [1]:
## autoreload 
%load_ext autoreload
%autoreload 2


In [2]:
from db.Materials import Material 
from db.Processes import Process

## helpers 
from db.db_helpers import (add_material, remove_material,update_material_name, clear_db,add_material_relationship)

## connectors & environment variables
from db.db_helpers import load_env_vars
from db.connector import DBConnector

In [3]:
## loading environment variables to connect to the database 
env_vars = load_env_vars()
## instantiate the connector 
connector = DBConnector(**env_vars)
## start the connection 
connector._connect()
## close the connection 
#connector._close()
## getting all the Material nodes
Material.nodes.all()

Loaded environment variables
Connected to the database


[<Material: {'uuid': 'M99', 'name': 'K', 'quantity': 100, 'unit': 'ton', 'cost': 1000000000, 'description': 'Material 99 - is pure Silver', 'id': 2}>,
 <Material: {'uuid': 'M98', 'name': 'Ur', 'quantity': 999, 'unit': 'ton', 'cost': 99999999999, 'description': "Material 98 - is pure Uranium, used in Max's Laboratory", 'id': 3}>]

In [4]:
hg_dict = {"uuid": "M99",
                 "name": "Hg",
                 "quantity": 100, "unit": "ton",
                 "cost": 1000000000,
                 "description":"Material 99 - is pure Silver"} ## the dictionary for the first material

hg,status = add_material(hg_dict)


ur_dict = {"uuid": "M98",
                 "name": "Ur",
                 "quantity": 999, "unit": "ton",
                 "cost": 99999999999,
                 "description":"Material 98 - is pure Uranium, used in Max's Laboratory"} ## the dictionary for the first material

ur,status2 = add_material(ur_dict)

print(Material.nodes.all(),end='\n')
clear_db()

Material: M99-Hg added
Material: M98-Ur added
[<Material: {'uuid': 'M99', 'name': 'K', 'quantity': 100, 'unit': 'ton', 'cost': 1000000000, 'description': 'Material 99 - is pure Silver', 'id': 2}>, <Material: {'uuid': 'M98', 'name': 'Ur', 'quantity': 999, 'unit': 'ton', 'cost': 99999999999, 'description': "Material 98 - is pure Uranium, used in Max's Laboratory", 'id': 3}>, <Material: {'uuid': 'M99', 'name': 'Hg', 'quantity': 100, 'unit': 'ton', 'cost': 1000000000, 'description': 'Material 99 - is pure Silver', 'id': 4}>, <Material: {'uuid': 'M98', 'name': 'Ur', 'quantity': 999, 'unit': 'ton', 'cost': 99999999999, 'description': "Material 98 - is pure Uranium, used in Max's Laboratory", 'id': 5}>]
Database cleared


([], [])

In [5]:
clear_db()
## Silver
hg,status = add_material(hg_dict)
## Uranium 
ur,status2 = add_material(ur_dict)
## create a relationship 
rel_status1 = add_material_relationship(hg, ur) ## hg --> ur 
rel_status2 = add_material_relationship(ur, hg) ## ur --> hg 

## update the material name 
old_uuid = "M99"
new_name = 'K'
upname1 = update_material_name(material_id=old_uuid, new_name=new_name)

Database cleared
Material: M99-Hg added
Material: M98-Ur added
Added relationship between M99 and M98
Added relationship between M98 and M99
Updated Material: M99 with new name: K


In [6]:
## add processes 
process_dict = {"uuid": "P1",
                "name": "Process 1",
                "materials_used": "M99",
                "output_of_process": "M98",
                "description": "Process 1 - is used to produce K",
                "total_input_cost": 1000000000,
                "total_output_cost": -99999
                }

p1 = Process(**process_dict).save()

In [7]:
## connect a material as input for the process
p1.input_material.connect(hg)

True

In [8]:
## connect the process as output for a material
p1.output_material.connect(ur)

True

In [ ]:
## add relationships of the processes. 

## the input will be Material99

## the output will be Material98

In [ ]:
#driver = GraphDatabase.driver(URL, auth=(username, password))
# neo_url = "http://localhost:7474"
# driver = GraphDatabase.driver(neo_url, auth=(username, password))

# with driver.session() as sess:
    
    
    

In [ ]:
# from neo4j import GraphDatabase

# driver = GraphDatabase.driver(URL, auth=(username, password))

# def create_friend_of(tx, name, friend):
#     tx.run("MATCH (a:Person) WHERE a.name = $name "
#            "CREATE (a)-[:KNOWS]->(:Person {name: $friend})",
#            name=name, friend=friend)

# with driver.session() as session:
#     session.write_transaction(create_friend_of, "Alice", "Bob")

# with driver.session() as session:
#     session.write_transaction(create_friend_of, "Alice", "Carl")

# driver.close()